In [26]:
import logging
import json
import torch

from vit_pytorch import ViT
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from collections import OrderedDict
from typing import Tuple

x = torch.randn(size=(4,3,180,320))
x.shape

torch.Size([4, 3, 180, 320])

In [8]:
conv1 = torch.nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding='same')
conv2 = torch.nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding='same')
conv3 = torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding='same')
conv4 = torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding='same')

pool_heat = torch.nn.Conv2d(in_channels=64,out_channels=1,kernel_size=1,padding='same')       
relu = torch.nn.ReLU()
max_pool = torch.nn.MaxPool2d(kernel_size=2,stride=2)


In [27]:
out1 = max_pool(relu(conv1(x)))
out1.shape

torch.Size([4, 16, 90, 160])

In [28]:
out2 = max_pool(relu(conv2(out1)))
out2.shape

torch.Size([4, 32, 45, 80])

In [30]:
out3 = max_pool(relu(conv3(out2)))
out3.shape

torch.Size([4, 64, 22, 40])

In [31]:
pool_heat(out3).shape

torch.Size([4, 1, 22, 40])

### pool out 3

In [65]:
pool3_heat_in = pool_heat(out3)
heat3_in = pool3_heat_in.reshape(-1,4,np.prod([*out3.shape[-2:]]))
heat3_in.shape

torch.Size([1, 4, 880])

In [83]:
lstm = torch.nn.LSTM(
    input_size=heat3_in.shape[-1],
    hidden_size=heat3_in.shape[-1],
    num_layers=1,
    batch_first=True,
    bidirectional=False,
)
h0 = torch.zeros(
    1,
    heat3_in.size(0),
    heat3_in.shape[-1],
    device=x.device
).requires_grad_()

# Initialize cell state
c0 = torch.zeros(
    1,
    heat3_in.size(0),
    heat3_in.shape[-1],
    device=x.device
).requires_grad_()
heat3_out , hidden_state = lstm(heat3_in,(h0,c0))
heat3_out.shape


torch.Size([1, 4, 880])

In [84]:
heat3_out = heat3_out.reshape(-1,*out3.shape[-2:],1)
heat3_out.shape

torch.Size([4, 22, 40, 1])

In [85]:
pool3_heat_in = pool_heat(out3)
pool3_heat_in.shape

torch.Size([4, 1, 22, 40])

In [91]:
b = pool3_heat_in.reshape(1,4,22,40,1)[:,4-3,:,:,:]
b.shape

torch.Size([1, 22, 40, 1])

In [92]:
pool3_heat_out = torch.add(heat3_out,b)
pool3_heat_out.shape


torch.Size([4, 22, 40, 1])

In [94]:
out4 = max_pool(relu(conv4(out3)))
out4.shape

torch.Size([4, 64, 11, 20])

### pool out4

In [95]:
pool4_heat_in = pool_heat(out4)
heat4_in = pool4_heat_in.reshape(-1,4,np.prod([*out4.shape[-2:]]))
heat4_in.shape

torch.Size([1, 4, 220])

In [96]:
lstm1 = torch.nn.LSTM(
    input_size=heat4_in.shape[-1],
    hidden_size=heat4_in.shape[-1],
    num_layers=1,
    batch_first=True,
    bidirectional=False,
)
h01 = torch.zeros(
    1,
    heat4_in.size(0),
    heat4_in.shape[-1],
    device=x.device
).requires_grad_()

# Initialize cell state
c01 = torch.zeros(
    1,
    heat4_in.size(0),
    heat4_in.shape[-1],
    device=x.device
).requires_grad_()
heat4_out , hidden_state = lstm1(heat4_in,(h01,c01))
heat4_out.shape

torch.Size([1, 4, 220])

In [105]:
heat4_out = heat4_out.reshape(-1,*out4.shape[-2:],1)
heat4_out.shape

torch.Size([16, 11, 20, 1])

In [106]:
b1 = pool4_heat_in.reshape(1,4,*out4.shape[-2:],1)[:,4-3,:,:,:]
b1.shape

torch.Size([1, 11, 20, 1])

In [107]:
pool4_heat_out = torch.add(heat4_out,b1)
pool4_heat_out.shape

torch.Size([16, 11, 20, 1])

### pool out5

In [100]:
out5 = max_pool(relu(conv4(out4)))
out5.shape

torch.Size([4, 64, 5, 10])

In [101]:
pool5_heat_in = pool_heat(out5)
heat5_in = pool5_heat_in.reshape(-1,4,np.prod([*out5.shape[-2:]]))
heat5_in.shape

torch.Size([1, 4, 50])

In [103]:
lstm2 = torch.nn.LSTM(
    input_size=heat5_in.shape[-1],
    hidden_size=heat5_in.shape[-1],
    num_layers=1,
    batch_first=True,
    bidirectional=False,
)
h02 = torch.zeros(
    1,
    heat5_in.size(0),
    heat5_in.shape[-1],
    device=x.device
).requires_grad_()

# Initialize cell state
c02 = torch.zeros(
    1,
    heat5_in.size(0),
    heat5_in.shape[-1],
    device=x.device
).requires_grad_()
heat5_out , hidden_state = lstm2(heat5_in,(h02,c02))
heat5_out.shape

torch.Size([1, 4, 50])

In [104]:
heat5_out = heat5_out.reshape(-1,*out5.shape[-2:],1)
heat5_out.shape

torch.Size([4, 5, 10, 1])

In [108]:
b2 = pool5_heat_in.reshape(1,4,*out5.shape[-2:],1)[:,4-3,:,:,:]
b2.shape

torch.Size([1, 5, 10, 1])

In [109]:
pool5_heat_out = torch.add(heat5_out,b2)
pool5_heat_out.shape

torch.Size([4, 5, 10, 1])

### pool5 upsampling



In [ ]:
import torch.nn.functional as F
'''

input – input tensor of shape (minibatch,in_channels,iH,iW)(minibatch,in_channels,iH,iW)

weight – filters of shape (in_channels,out_channelsgroups,kH,kW)(in_channels,groupsout_channels​,kH,kW)

bias – optional bias of shape (out_channels)(out_channels). Default: None

stride – the stride of the convolving kernel. Can be a single number or a tuple (sH, sW). Default: 1

padding – dilation * (kernel_size - 1) - padding zero-padding will be added to both sides of each dimension in the input. Can be a single number or a tuple (padH, padW). Default: 0

output_padding – additional size added to one side of each dimension in the output shape. Can be a single number or a tuple (out_padH, out_padW). Default: 0

groups – split input into groups, in_channelsin_channels should be divisible by the number of groups. Default: 1

dilation – the spacing between kernel elements. Can be a single number or a tuple (dH, dW). Default: 1

'''
y_up_tor = F.conv_transpose(y_tor, feats_tor, output_padding=(1,1), stride=(2,2))

In [33]:
import numpy as np

out = np.load('/data/humanoid_train_data/im_hm_interact_10.npz')
im,hm,interact = out['img'],out['hm'],out['interact']
im.shape

(4, 180, 320, 3)